In [1]:
import re
import os
import operator
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter

In [2]:
all_files = os.listdir("subs")
path = "subs\\"

In [3]:
new_df = pd.DataFrame([])
for file in tqdm(all_files):
    file_path = os.path.join(path, file)
    df = pd.read_csv(file_path)
    df["file_name"] = file
    new_df = pd.concat([new_df, df])

In [4]:
# Removing duplicates

dedup_df = new_df.drop_duplicates()
dedup_df = dedup_df.reset_index()

# Removing mappings low quality semantic score 

dedup_df = dedup_df[dedup_df['semantic_score'] > 0.01]
dedup_df = dedup_df[operator.and_(dedup_df['semantic_score'] > 0.2,  dedup_df['score'] > 0.2)]

# Removing en_text with less than 3 characters

dedup_df = dedup_df[dedup_df['en_text'].map(str).map(len) > 3]

# Remove words inside {} and <> in jp_text and en_text

dedup_df['jp_text'] = [re.sub(r"{.*}", "", x) for x in dedup_df['jp_text']]
dedup_df['en_text'] = [re.sub(r"{.*}", "", x) for x in dedup_df['en_text']]
dedup_df['jp_text'] = [re.sub(r"<.*>", "", x) for x in dedup_df['jp_text']]
dedup_df['en_text'] = [re.sub(r"<.*>", "", x) for x in dedup_df['en_text']]
dedup_df['jp_text'] = [re.sub(r"\（.*\）", "", x) for x in dedup_df['jp_text']]

# Replacing \\N with \n in en_texts
dedup_df['en_text'] = dedup_df['en_text'].str.replace(r"\\N", "\n")

In [5]:
#dedup_df = pd.read_csv("dedup_df.csv")
dedup_df["file_name"] = dedup_df["file_name"].str.replace(".csv","")

In [6]:
animes = []
for name in tqdm(dedup_df["file_name"]):
    animes.append(re.sub(r'-[0-9]+', "", name))

In [7]:
files = []
for name in tqdm(all_files):
    name = name.replace(".csv","")
    files.append(re.sub(r'-[0-9]+', "", name))

In [8]:
new_dict = {}
for key in tqdm(Counter(animes)):
    try:
        new_dict[key] = Counter(animes)[key]/Counter(files)[key]
    except:
        continue

In [9]:
new_dict = {k: v for k, v in sorted(new_dict.items(), key=lambda item: item[1])}

In [10]:
new_dict

{'Pupa': 6.636363636363637,
 'Space_Brothers': 17.90721649484536,
 'Dragon_Ball_Super': 21.671755725190838,
 'Haikyuu!!_S3': 22.2,
 'Fairy_Tail': 24.97714285714286,
 'The_Devil_Is_a_Part-Timer': 28.53846153846154,
 'Diabolik_Lovers_More_Blood': 28.666666666666668,
 'Noragami': 29.75,
 'Sekirei_Pure_Engagement': 30.307692307692307,
 'Ghost_in_the_Shell': 31.0,
 'Lucky_Star': 31.708333333333332,
 'One-Punch_Man': 32.0,
 'Aggretsuko': 32.5,
 'Kaiji_S1': 33.19230769230769,
 'Haikyuu!!': 34.16,
 'Tokyo_Ghoul_Root': 34.5,
 'Haikyuu!!_S2': 34.52,
 'Mushibugyo': 36.76923076923077,
 'Owari_no_Seraph': 37.083333333333336,
 'Kaiji_S2': 38.36,
 'Sakurasou_no_Pet_na_Kanojo': 38.791666666666664,
 'Tales_of_Zestiria_the_X': 41.166666666666664,
 'Noragami_Aragoto': 41.30769230769231,
 'Inuyashiki': 41.45454545454545,
 'Haganai_Next': 41.583333333333336,
 'Kimi_to_Boku': 41.92307692307692,
 'Aldnoah_Zero': 42.333333333333336,
 'Ano_Natsu_de_Matteru': 42.333333333333336,
 'Ping_Pong_The_Animation': 42.3

In [75]:
dedup_df.to_csv("dedup_df.csv",index=False)